# Программирование на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум по сбору информации о курсах валют
### Часть 1: выгружаем данные

Загружаем необходимые модули и библиотеки:

In [2]:
import requests
from bs4 import BeautifulSoup

Отправляем запрос к странице с курсами валют от РБК (обновляется, результаты могут отличаться в зависимости от времени подключения), забираем исходный код HTML и превращаем его в объект BeautifulSoup:

In [3]:
page = requests.get("https://cash.rbc.ru/")
soup = BeautifulSoup(page.text)

Несмотря на то, что курсы валют в разных банках представлены в табличном виде, настоящей таблицей с тэгом `<table>` этот фрагмент страницы не является. Но это нестрашно, будем искать информацию по разделам с тэгами `<div>`. Найдем все разделы с подходящим классом и сохраним их в список `divs`, это будут «карточки» с информацией о курсе валют в разных банках:

In [4]:
divs = soup.find_all("div",
                     {"class" : "quote__office__one js-one-office"})

Сначала научимся извлекать информацию на примере одного банка:

In [12]:
d = divs[1]

In [13]:
print(d)

<div class="quote__office__one js-one-office">
<div class="quote__office__cell quote__office__one__star">
<div class="js-select-favorites"></div>
</div>
<div class="quote__office__cell">
<a class="quote__office__one__name" href="/cash/bank/103134.html">Сбербанк ДО № 9038/01698</a>
<div class="quote__office__one__phone">+7 800 555-55-50</div>
</div>
<div class="quote__office__cell quote__office__one__button_full">
</div>
<div class="quote__office__cell quote__office__one__rate quote__mode_list_view">
            83.45
        </div>
<div class="quote__office__cell quote__office__one__cash quote__mode_list_view">
            8345.00
        </div>
<div class="quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro">
            78.79
                <div class="quote__office__one__diapason">от </div>
</div>
<div class="quote__office__cell quote__office__one__buy js-filter-pro quote__mode_list_view_pro">
            83.45
                <div class="quote__off

Плюс автоматической выгрузки данных: в исходном коде страницы хранится больше информации, чем мы видим непосредственно как пользователи. В выгруженных «карточках» есть не только информация о покупке валюты, которая отображается на сайте по умолчанию, но и курс при продаже, а также ближайшая к банку станция метро с расстоянием до него.

Найдем ссылку на страницу банка и заберем из нее две части – саму ссылку (`link_href`) и название банка (`link_text`):

In [14]:
link = d.find("a", {"class" : "quote__office__one__name"})
link_href = link.get("href")
link_text = link.text
print(link_href, link_text)

/cash/bank/103134.html Сбербанк ДО № 9038/01698


Аналогичным образом извлечем телефон банка, курс валют при покупке и при продаже (попутно избавимся от лишних отступов и слов в курсах валют):

In [15]:
phone = d.find("div", {"class" : "quote__office__one__phone"}).text

In [16]:
buy = d.find("div", 
             {"class" : "quote__office__cell quote__office__one__rate quote__mode_list_view"}).text
buy = buy.strip()

sell = d.find("div", 
              {"class" : "quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro"}).text
sell = sell.replace("от", "").strip()

In [17]:
print(phone, buy, sell)

+7 800 555-55-50 83.45 78.79


Пока все значения оставим в виде строк (тип `str`), преобразование типов выполним в самом конце, когда соберем всю информацию и сохраним в датафрейм.

Теперь найдем ближайшую станцию метро и расстояние до нее (в первом случае тэг `<span>` не имеет атрибута `class`, это его отличительная особенность, поэтому класс будет пустой):

In [18]:
metro = d.find("span", {"class" : None}).text
distance = d.find("span", 
                  {"class" : "quote__office__metro__distance"}).text
print(metro, distance)

Южная 481 м


Отлично! Всю информацию по одному банку выгружать мы научились. Напишем функцию, чтобы применить ее потом в цикле ко всем «карточкам» банков в `divs`. Собственно, для написания функции нам достаточно скопировать блоки кода выше и перечислить в `return` итоговые результаты в виде кортежа:

In [19]:
def get_bank(d):
    link = d.find("a", {"class" : "quote__office__one__name"})
    link_href = link.get("href")
    link_text = link.text
    phone = d.find("div", {"class" : "quote__office__one__phone"}).text
    buy = d.find("div", 
                 {"class" : "quote__office__cell quote__office__one__rate quote__mode_list_view"}).text
    buy = buy.strip()
    sell = d.find("div", 
                  {"class" : "quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro"}).text
    sell = sell.replace("от", "").strip()
    
    metro = d.find("span", {"class" : None}).text
    distance = d.find("span", 
                  {"class" : "quote__office__metro__distance"}).text
    return link_href, link_text, phone, buy, sell, metro, distance

Попробуем применить функцию ко всем элементам списка `divs`. Давайте напишем цикл с конструкцией `try-except`, чтобы если что, зафиксировать случаи, когда функция не срабатывает из-за ошибок:

In [20]:
banks_info = []
errors = []

for bank in divs:
    try:
        res = get_bank(bank)
        banks_info.append(res)
    except:
        errors.append(bank)

Проверим, сколько у нас банков и сколько ошибок:

In [21]:
print(len(banks_info), len(errors))

3 2


Список с фрагментами кода из `divs`, на которых функция не сработала, оказался непустым. Если мы изучим его элементы, окажется, что это те банки, у которых нет метро поблизости (станция и расстояние не указаны). Скорректируем нашу функцию на случай, если информация о метро отсутствует – добавим конструкцию `try-except`:

In [22]:
def get_bank(d):
    link = d.find("a", {"class" : "quote__office__one__name"})
    link_href = link.get("href")
    link_text = link.text
    phone = d.find("div", {"class" : "quote__office__one__phone"}).text
    buy = d.find("div", {"class" : "quote__office__cell quote__office__one__rate quote__mode_list_view"}).text
    buy = buy.strip()
    sell = d.find("div", {"class" : "quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro"}).text
    sell = sell.replace("от", "").strip()
    try:
        metro = d.find("span", {"class" : None}).text
        distance = d.find("span", 
                  {"class" : "quote__office__metro__distance"}).text
    except:
        metro = None
        distance= None
    return link_href, link_text, phone, buy, sell, metro, distance

Повторяем выгрузку:

In [23]:
banks_info = []
errors = []

for bank in divs:
    try:
        res = get_bank(bank)
        banks_info.append(res)
    except:
        errors.append(bank)

In [24]:
print(len(banks_info), len(errors))

5 0


Теперь все отлично! Осталось превратить все в датафрейм `pandas` и преобразовать данные.

### Часть 2: обрабатываем данные

Импортируем библиотеку `pandas`:

In [25]:
import pandas as pd

Получаем датафрейм из списка кортежей:

In [26]:
df = pd.DataFrame(banks_info)

Приписываем названия столбцам:

In [27]:
df.columns = ["link", "name", "phone", 
              "buy", "sell", "metro", "distance"]

Сделаем ссылки на страницы банков абсолютными – доклеим к каждой ссылку на главную страницу. Чтобы избежать циклов, опишем нужное преобразование с помощью анонимной lambda-функции и применим ее ко всем элементам столбца `link` через метод `.apply()`:

In [28]:
df["link"] = df["link"].apply(lambda x: "https://cash.rbc.ru" + x)

Пока все столбцы в датафрейме у нас текстовые – в `pandas` вместо `string` текстовый тип назвается `object`:

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      5 non-null      object
 1   name      5 non-null      object
 2   phone     5 non-null      object
 3   buy       5 non-null      object
 4   sell      5 non-null      object
 5   metro     3 non-null      object
 6   distance  3 non-null      object
dtypes: object(7)
memory usage: 408.0+ bytes


Скорректируем типы столбцов с курсами валют – переделаем их на `float`:

In [30]:
df["buy"] = df["buy"].astype(float)
df["sell"] = df["sell"].astype(float)
df.head(10)

,link,name,phone,buy,sell,metro,distance
0,https://cash.rbc.ru/cash/bank/103237.html,КБ ЮНИСТРИМ ОО № 154 Садовод,+7 989 048-11-35,82.90,81.75,None,None
1,https://cash.rbc.ru/cash/bank/103134.html,Сбербанк ДО № 9038/01698,+7 800 555-55-50,83.45,78.79,Южная,481 м
2,https://cash.rbc.ru/cash/bank/102664.html,Банк БКФ ДО На Проспекте Мира,+7 915 110-05-40,83.50,81.00,Алексеевская,709 м
3,https://cash.rbc.ru/cash/bank/102665.html,Банк БКФ ДО На Сущевском Валу,+7 915 022-53-67,83.50,81.00,Марьина Роща,276 м
4,https://cash.rbc.ru/cash/bank/103295.html,КБ Солидарность ДО Жуковский,+7 495 663-35-77 доб. 5399,85.00,80.50,None,None


Теперь разберемся с расстоянием до метро. 

In [31]:
df["distance"]

0     None
1    481 м
2    709 м
3    276 м
4     None
Name: distance, dtype: object

Где-то расстояния указаны в метрах, где-то в километрах. Давайте напишем функцию, которая определяет по строке единицы измерения и возвращает расстояния до метро в метрах:

In [32]:
def get_dist(x):
    try:
        if "км" in x:
            km = float(x.split()[0])
            m = km * 1000
        elif "м" in x:
            m = int(x.split()[0])
    except:
        m = None
    return m

Применяем эту функцию к столбцу `distance` и создаем новый столбец `distance_m`:

In [33]:
df["distance_m"] = df["distance"].apply(get_dist)
df

,link,name,phone,buy,sell,metro,distance,distance_m
0,https://cash.rbc.ru/cash/bank/103237.html,КБ ЮНИСТРИМ ОО № 154 Садовод,+7 989 048-11-35,82.90,81.75,None,None,NaN
1,https://cash.rbc.ru/cash/bank/103134.html,Сбербанк ДО № 9038/01698,+7 800 555-55-50,83.45,78.79,Южная,481 м,481.0
2,https://cash.rbc.ru/cash/bank/102664.html,Банк БКФ ДО На Проспекте Мира,+7 915 110-05-40,83.50,81.00,Алексеевская,709 м,709.0
3,https://cash.rbc.ru/cash/bank/102665.html,Банк БКФ ДО На Сущевском Валу,+7 915 022-53-67,83.50,81.00,Марьина Роща,276 м,276.0
4,https://cash.rbc.ru/cash/bank/103295.html,КБ Солидарность ДО Жуковский,+7 495 663-35-77 доб. 5399,85.00,80.50,None,None,NaN


Уберем старый столбец с расстоянием в текстовом виде. Удалим его с помощью метода `.drop()` и добавим внутри аргумент `inplace=True`, чтобы изменения сохранились в самом датафрейме.

In [34]:
# inplace = True, чтобы не писать
# df = df.drop(columns = ["distance"])

df.drop(columns = ["distance"], inplace = True)

In [35]:
df

,link,name,phone,buy,sell,metro,distance_m
0,https://cash.rbc.ru/cash/bank/103237.html,КБ ЮНИСТРИМ ОО № 154 Садовод,+7 989 048-11-35,82.90,81.75,None,NaN
1,https://cash.rbc.ru/cash/bank/103134.html,Сбербанк ДО № 9038/01698,+7 800 555-55-50,83.45,78.79,Южная,481.0
2,https://cash.rbc.ru/cash/bank/102664.html,Банк БКФ ДО На Проспекте Мира,+7 915 110-05-40,83.50,81.00,Алексеевская,709.0
3,https://cash.rbc.ru/cash/bank/102665.html,Банк БКФ ДО На Сущевском Валу,+7 915 022-53-67,83.50,81.00,Марьина Роща,276.0
4,https://cash.rbc.ru/cash/bank/103295.html,КБ Солидарность ДО Жуковский,+7 495 663-35-77 доб. 5399,85.00,80.50,None,NaN
